In [3]:
import sys

sys.version_info

sys.version_info(major=3, minor=9, micro=16, releaselevel='final', serial=0)

In [5]:
import tensorflow as tf

tf.__version__

2023-02-16 02:35:23.442251: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-16 02:35:23.617330: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-16 02:35:23.617378: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-16 02:35:24.647274: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

'2.11.0'

In [7]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 52.7 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 13.1 MB/s eta 0:00:00


In [8]:
import boto3

boto3.__version__

'1.26.72'

In [9]:
!pip install easydict

  Preparing metadata (setup.py) ... done
  Created wheel for easydict: filename=easydict-1.10-py3-none-any.whl size=6509 sha256=b1547c43c349a841adbb73d6a0bbcfb712bedc400ec46a0dc5ab89e42b2f435d
  Stored in directory: /root/.cache/pip/wheels/0d/9a/a9/02f3a5f0c6b2c57184661770360c58db8166f5c877780e98f2
Successfully built easydict


In [10]:
adict = dict()
adict['abd'] = 'ABC'

adict['abd']

'ABC'

In [12]:
class Sample:
    abc = 'ABD'
    
Sample.abc

'ABD'

In [13]:
from easydict import EasyDict

adict = EasyDict()
adict.abc = 'ABC'
adict.abc

'ABC'

In [15]:
settings = EasyDict()

# 아래 내용 반드시 지울 것
settings.AWS_ACCESS_KEY_ID = ""     # AWS 자격증명 ID
settings.AWS_SECRET_ACCESS_KEY = "" # AWS 자격증명 PW
settings.AWS_REGION_NAME = ""       # AWS 지역
settings.AWS_ACCOUNT_ID = ""        # AWS 계정 AD

In [17]:
import boto3

In [36]:
s3_client = boto3.client(
    service_name = 's3',
    aws_access_key_id = settings.AWS_ACCESS_KEY_ID,
    aws_secret_access_key = settings.AWS_SECRET_ACCESS_KEY,
    region_name = settings.AWS_REGION_NAME
)

s3_client

In [ ]:
settings.AWS_BUCKET_NAME = 'genia-bucket'
obj = s3_client.get_object(Bucket = settings.AWS_BUCKET_NAME, Key = 'raw/abalone.csv' )
obj

In [38]:
import pandas as pd

obj['Body']

In [39]:
res = pd.read_csv(obj['Body'], encoding = 'UTF8')

In [40]:
res

,Type,LongestShell,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [42]:
go = res.head()

In [45]:
import os  
os.makedirs('hw/', exist_ok=True)  
go.to_csv('hw/out.csv')  

In [46]:
response = s3_client.put_object(
    Body = 'hw/out.csv',
    Bucket = settings.AWS_BUCKET_NAME,
    Key = 'upload-test/student1_example.csv'
)

In [48]:
df = res[['Type', 'Diameter', 'Height', 'Rings']]

In [49]:
df['Type'].

array(['M', 'F', 'I'], dtype=object)

In [51]:
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 29.1 MB/s eta 0:00:0000:0100:01


In [52]:
df.to_parquet("hw/out.parquet", engine="pyarrow", compression="gzip")

In [53]:
response = s3_client.put_object(
    Body = 'hw/out.parquet',
    Bucket = settings.AWS_BUCKET_NAME,
    Key = 'upload-test/student1.parquet'
)

In [54]:
response

{'ResponseMetadata': {'RequestId': 'W7WX4QZJSBQV6RQE',
  'HostId': 'A7P+056kB9M2Fwg5BCcv8KWaPelL59hTzbMvG7Eek9pMloXnI9DMdMF8uZ0XBumBq0k2TLxO9K0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'A7P+056kB9M2Fwg5BCcv8KWaPelL59hTzbMvG7Eek9pMloXnI9DMdMF8uZ0XBumBq0k2TLxO9K0=',
   'x-amz-request-id': 'W7WX4QZJSBQV6RQE',
   'date': 'Thu, 16 Feb 2023 06:10:46 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"72f105c12910a9abebfa9d6ce65f818c"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"72f105c12910a9abebfa9d6ce65f818c"',
 'ServerSideEncryption': 'AES256'}

In [55]:
!pip install smart_open

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.6 MB/s eta 0:00:00


In [56]:
# 기존 open 함수가 있기 때문에 이름을 다르게 지정해 주기로 한다.
from smart_open import open as s_open

# 아래처럼 변수에 저장된 정보들이 byte 형태다...
df

,Type,Diameter,Height,Rings
0,M,0.365,0.095,15
1,M,0.265,0.090,7
2,F,0.420,0.135,9
3,M,0.365,0.125,10
4,I,0.255,0.080,7
...,...,...,...,...
4172,F,0.450,0.165,11
4173,M,0.440,0.135,10
4174,M,0.475,0.205,9
4175,F,0.485,0.150,10


In [58]:
file_name = f"S3://{settings.AWS_BUCKET_NAME}/upload-test/student1.parquet"
with s_open(file_name, "wb", transport_params = dict(client = s3_client)) as out_file:
    df.to_parquet(out_file, engine = 'pyarrow', compression = 'gzip', index = False)